# Big Data Econometrics HW1 - Synthetic OLS with Bootstrapping

In [18]:
%pip install numpy pandas matplotlib scikit-learn statsmodels

1598.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/93/fb/9af1082bec870188c42a1c239839915b74a5099c392389ff04215dcee812/numpy-2.3.3-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 835.5 kB/s eta 0:00:0000:0100:01
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/38/18/48f10f1cc5c397af59571d638d211f494dba481f449c19adbd282aa8f4ca/pandas-2.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/8c/af/30ddefe19ca67eebd70047dabf50f899eaff6f3c5e6a1a7edaecaf63f794/matplotlib-3.10.6-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packa

In [17]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
np.random.seed(42) 


ModuleNotFoundError: No module named 'numpy'

Generating Synthetic Data (10,000 rows; 2 Normal, 2 Uniform)

In [ ]:

from numpy.random import default_rng
rng = default_rng(42)
n = 10_000

x1 = rng.normal(0, 1, n)
x2 = rng.normal(1.0, 2.0, n) 

x3 = rng.uniform(-2.0, 2.0, n)
x4 = rng.uniform(0.0, 5.0, n)

x1_sq = x1**2
eps = rng.normal(0, 0.5, n) 

beta_true = {"const": 1.25, "x1": 3.0, "x2": -2.0, "x3": 0.5, "x4": 1.5, "x1_sq": 0.8}

y = (beta_true["const"] + beta_true["x1"]*x1 + beta_true["x2"]*x2 + beta_true["x3"]*x3
     + beta_true["x4"]*x4 + beta_true["x1_sq"]*x1_sq + eps)

df = pd.DataFrame({"x1": x1, "x2": x2, "x3": x3, "x4": x4, "x1_sq": x1_sq, "y": y})
df.head()


Test/Train Split (70/30)

In [ ]:

features = ["x1", "x2", "x3", "x4", "x1_sq"]
X = df[features].copy()
y_all = df["y"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.30, random_state=42)
X_train.shape, X_test.shape


In [ ]:

X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

ols_model = sm.OLS(y_train, X_train_sm).fit()
yhat_train = ols_model.predict(X_train_sm)
yhat_test = ols_model.predict(X_test_sm)

mse_train = mean_squared_error(y_train, yhat_train)
mse_test = mean_squared_error(y_test, yhat_test)

print(ols_model.summary())
print("\nTrain MSE:", round(mse_train, 4))
print("Test  MSE:", round(mse_test, 4))


### Residual Diagnostics (Train)

In [ ]:
plt.figure()
plt.scatter(yhat_train, y_train - yhat_train, alpha=0.3)
plt.axhline(0, linestyle="--")
plt.xlabel("Fitted values (train)")
plt.ylabel("Residuals (train)")
plt.title("Residuals vs Fitted (Train)")
plt.show()

plt.figure()
plt.hist(y_train - yhat_train, bins=40)
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Residuals Histogram (Train)")
plt.show()


## Bootstrap Resamples, OLS for Each

In [ ]:

B = 10
coef_list = []
for b in range(B):
    idx = rng.integers(0, len(df), size=len(df))
    X_b = sm.add_constant(df.loc[idx, features])
    y_b = df.loc[idx, "y"].values
    model_b = sm.OLS(y_b, X_b).fit()
    coef_list.append(model_b.params)

boot_coef_df = pd.DataFrame(coef_list)
boot_coef_df.index.name = "bootstrap_id"
boot_coef_df


In [ ]:

param_stats = pd.DataFrame({"mean": boot_coef_df.mean(axis=0),
                            "std": boot_coef_df.std(axis=0, ddof=1)})
param_stats


In [ ]:

ols_params = ols_model.params.rename("train_OLS_coef")
comp = pd.concat([ols_params, param_stats], axis=1)
comp.round(4)


### Interpretation

With a correctly specified linear model (including the x1^2 term) and a large n=10,000, the OLS
coefficients are estimated very precisely. The bootstrap means are close to the train OLS point
estimates, and the bootstrap standard deviations are small, indicating low sampling variability.
Train and test MSE are very similar, consistent with low overfitting.

In [ ]:

!apt-get install git -y

!git config --global user.email "bleasee@bc.edu"
!git config --global user.name "Elsa Blease"


In [ ]:
!git clone https://github.com/elsablease/MyRepository.git


In [ ]:
!cp /content/ADEC7430HW1.ipynb /content/MyRepository/


In [ ]:
%cd /content/MyRepository

!git add .
!git commit -m "Add Big Data Econ Assignment with bootstrap analysis"
!git push origin main
